# Animas Salt Optimization

## Import libraries

In [58]:
import sys
import os
import numpy as np
import pandas as pd
import flopy as flopy
import pyemu
import shapefile #the pyshp module
import matplotlib.pyplot as plt
import glob
import shutil

In [59]:
from apexmf import apexmf_pst_utils, apexmf_pst_par, apexmf_utils
import apexmf
apexmf.__version__

'0.2.0'

In [60]:
# sys.path.insert(0, 'D:/spark-brc_gits/apexmf_git/apexmf')
# import apexmf_pst_par, apexmf_pst_utils

# 1. Set up and write apexmf.con file

In [61]:
# working directory and file names
wd = "D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614"
# calibration period
sim_start_day = '1/1/1987'
cali_start_day = '1/1/1992'
cali_end_day = '12/31/2011'
# time step
time_step = 'month'
salt_subs = [9, 12, 75]
os.chdir(wd)

In [62]:
con =  apexmf_pst_utils.create_apexmf_con(
                        wd, sim_start_day, cali_start_day, cali_end_day,
                        salt_subs=salt_subs,
                        time_step=time_step
                        )

In [63]:
con

,names,vals
0,wd,D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614
1,mfwd,D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614/MODFLOW
2,sim_start,1/1/1987
3,cal_start,1/1/1992
4,cal_end,12/31/2011
5,cha_file,n
6,subs,n
7,gw_level,n
8,grids,n
9,lai_file,n


# 2. Create Template file

## 2.1 MODFLOW (Skipped: already set) 

## 2.2 APEX | PARM (Skipped: already set) 

## 2.3 Salt Parameter

### 2.3.1 RT3D | initial salt concentration
- create new par file for rt3d_initial concentration and its template file
- extract original btn data and each initial concentration info and export it to each file (org)
- create reference file updated using new par file and orginal file
- update btn file

In [64]:
salt_pars = apexmf_pst_par.rt3d_initc_par()
salt_pars

'rt3d_initc.par' file has been exported to the MODFLOW working directory!


,parnme,chg_type,val
parnme,,,
init_cno3,init_cno3,pctchg,0.0001
init_p,init_p,pctchg,0.0001
init_so4,init_so4,pctchg,0.0001
init_ca,init_ca,pctchg,0.0001
init_mg,init_mg,pctchg,0.0001
init_na,init_na,pctchg,0.0001
init_k,init_k,pctchg,0.0001
init_cl,init_cl,pctchg,0.0001
init_co3,init_co3,pctchg,0.0001


In [65]:
# create initc template file
apexmf_pst_par.rt3d_initc_pars_tpl()

,parnme,chg_type,parval1,tpl
parnme,,,,
init_cno3,init_cno3,pctchg,0.0001,~ init_cno3 ~
init_p,init_p,pctchg,0.0001,~ init_p ~
init_so4,init_so4,pctchg,0.0001,~ init_so4 ~
init_ca,init_ca,pctchg,0.0001,~ init_ca ~
init_mg,init_mg,pctchg,0.0001,~ init_mg ~
init_na,init_na,pctchg,0.0001,~ init_na ~
init_k,init_k,pctchg,0.0001,~ init_k ~
init_cl,init_cl,pctchg,0.0001,~ init_cl ~
init_co3,init_co3,pctchg,0.0001,~ init_co3 ~


- ### extract original btn data and each initial concentration info and export it to each file (org)

In [22]:
apexmf_pst_par.extract_org_rt3d_cons() # extract initial condition as backup array file

The "rt3d_btn.org" file already exists...
'init_cno3.org' already exists ...
'init_p.org' already exists ...
'init_so4.org' already exists ...
'init_ca.org' already exists ...
'init_mg.org' already exists ...
'init_na.org' already exists ...
'init_k.org' already exists ...
'init_cl.org' already exists ...
'init_co3.org' already exists ...
'init_hco3.org' already exists ...


- ### update init rt3d ions and btn file

In [9]:
apexmf_pst_par.update_rt3d_ions_refs() # update ref array file using org and rt3d_initc.par

init_cno3.ref has been updated ...
init_p.ref has been updated ...
init_so4.ref has been updated ...
init_ca.ref has been updated ...
init_mg.ref has been updated ...
init_na.ref has been updated ...
init_k.ref has been updated ...
init_cl.ref has been updated ...
init_co3.ref has been updated ...
init_hco3.ref has been updated ...


In [10]:
apexmf_pst_par.update_btn() # replace old initc with updated ones

EM_Animas_RBF.btn file has been updated ...


### 2.3.2 SALINITY | initial salt concentration
- extract updated init salt concs. from rt3d and convert them to sub scale
- update salt input file

In [11]:
apexmf_pst_par.update_salt_input()

The "salt_input.org" file already exists...
Updated gridded init salt ions have been converted to sub scales ...
The "salt_input" file has been updated ...


# 3. Create instruction files
- we have month and daily 8 differenct ions.
- extract simulated salt outputs
- create each ins file matching simulated and observed data


In [23]:
# set variables
salt_subs = [9, 12, 75]
time_steps = ['day', 'mon']

In [24]:
salt_df = pd.read_csv(
                    "SALINITY/salt.output.channels",
                    delim_whitespace=True,
                    skiprows=4,
                    header=0,
                    index_col=0,
                    )
salt_df = salt_df.iloc[:, 5:] # only cols we need
colnams = salt_df.columns.tolist()
colnams

['load_so4',
 'load_ca',
 'load_mg',
 'load_na',
 'load_k',
 'load_cl',
 'load_co3',
 'load_hco3',
 'conc_so4',
 'conc_ca',
 'conc_mg',
 'conc_na',
 'conc_k',
 'conc_cl',
 'conc_co3',
 'conc_hco3']

In [25]:
colnams

['load_so4',
 'load_ca',
 'load_mg',
 'load_na',
 'load_k',
 'load_cl',
 'load_co3',
 'load_hco3',
 'conc_so4',
 'conc_ca',
 'conc_mg',
 'conc_na',
 'conc_k',
 'conc_cl',
 'conc_co3',
 'conc_hco3']

In [26]:
sim_files = []
for i in salt_subs:
    for j in colnams:
        for k in time_steps:
            sim_files.append("salt_{}_{:03d}_{}.txt".format(j, i, k))

In [27]:
sim_files

['salt_load_so4_009_day.txt',
 'salt_load_so4_009_mon.txt',
 'salt_load_ca_009_day.txt',
 'salt_load_ca_009_mon.txt',
 'salt_load_mg_009_day.txt',
 'salt_load_mg_009_mon.txt',
 'salt_load_na_009_day.txt',
 'salt_load_na_009_mon.txt',
 'salt_load_k_009_day.txt',
 'salt_load_k_009_mon.txt',
 'salt_load_cl_009_day.txt',
 'salt_load_cl_009_mon.txt',
 'salt_load_co3_009_day.txt',
 'salt_load_co3_009_mon.txt',
 'salt_load_hco3_009_day.txt',
 'salt_load_hco3_009_mon.txt',
 'salt_conc_so4_009_day.txt',
 'salt_conc_so4_009_mon.txt',
 'salt_conc_ca_009_day.txt',
 'salt_conc_ca_009_mon.txt',
 'salt_conc_mg_009_day.txt',
 'salt_conc_mg_009_mon.txt',
 'salt_conc_na_009_day.txt',
 'salt_conc_na_009_mon.txt',
 'salt_conc_k_009_day.txt',
 'salt_conc_k_009_mon.txt',
 'salt_conc_cl_009_day.txt',
 'salt_conc_cl_009_mon.txt',
 'salt_conc_co3_009_day.txt',
 'salt_conc_co3_009_mon.txt',
 'salt_conc_hco3_009_day.txt',
 'salt_conc_hco3_009_mon.txt',
 'salt_load_so4_012_day.txt',
 'salt_load_so4_012_mon.txt',


In [28]:
obd_files = []
for i in salt_subs:
    for k in time_steps:
        obd_files.append("salt_{:03d}_{}.obd".format(i, k))

In [29]:
obd_files

['salt_009_day.obd',
 'salt_009_mon.obd',
 'salt_012_day.obd',
 'salt_012_mon.obd',
 'salt_075_day.obd',
 'salt_075_mon.obd']

- ### extract simulated salts

In [20]:
apexmf_pst_utils.extract_salt_results(salt_subs, sim_start_day,cali_start_day, cali_end_day)

salt_load_so4_009_day.txt
salt_load_so4_009_mon.txt
salt_load_ca_009_day.txt
salt_load_ca_009_mon.txt
salt_load_mg_009_day.txt
salt_load_mg_009_mon.txt
salt_load_na_009_day.txt
salt_load_na_009_mon.txt
salt_load_k_009_day.txt
salt_load_k_009_mon.txt
salt_load_cl_009_day.txt
salt_load_cl_009_mon.txt
salt_load_co3_009_day.txt
salt_load_co3_009_mon.txt
salt_load_hco3_009_day.txt
salt_load_hco3_009_mon.txt
salt_conc_so4_009_day.txt
salt_conc_so4_009_mon.txt
salt_conc_ca_009_day.txt
salt_conc_ca_009_mon.txt
salt_conc_mg_009_day.txt
salt_conc_mg_009_mon.txt
salt_conc_na_009_day.txt
salt_conc_na_009_mon.txt
salt_conc_k_009_day.txt
salt_conc_k_009_mon.txt
salt_conc_cl_009_day.txt
salt_conc_cl_009_mon.txt
salt_conc_co3_009_day.txt
salt_conc_co3_009_mon.txt
salt_conc_hco3_009_day.txt
salt_conc_hco3_009_mon.txt
salt_load_so4_012_day.txt
salt_load_so4_012_mon.txt
salt_load_ca_012_day.txt
salt_load_ca_012_mon.txt
salt_load_mg_012_day.txt
salt_load_mg_012_mon.txt
salt_load_na_012_day.txt
salt_load_n

- ### create instruction files

In [21]:
for ss in salt_subs:
    for ts in time_steps:
        obd_file = "salt_{:03d}_{}.obd".format(ss, ts)
        # open obd_file and check if column exists
        obd_df = pd.read_csv(
                    os.path.join(wd, obd_file),
                    sep='\t',
                    index_col=0,
                    parse_dates=True,
                    na_values=[-999, '']
                        )
        for cn in colnams:
            if cn in obd_df.columns:
                salt_sim_file = "salt_{}_{:03d}_{}.txt".format(cn, ss, ts)
                apexmf_pst_utils.salt_obd_to_ins(
                    ss,
                    os.path.join(wd, salt_sim_file), 
                    os.path.join(wd, obd_file), 
                    cn, sim_start_day, cali_end_day, time_step=ts)

D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_load_so4_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_load_ca_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_load_mg_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_load_na_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_load_k_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_conc_so4_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_conc_ca_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_conc_mg_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF_PS0G_cal_220614\salt_conc_na_009_day.txt.ins file has been created...
D:/Projects/Watersheds/Elizabeth/RBF

# 4. Create dummy control file pst
* Note: all ins tpl files should be located in main directory

In [7]:
io_files = pyemu.helpers.parse_dir_for_io_files('.')
pst = pyemu.Pst.from_io_files(*io_files)
pyemu.helpers.pst_from_io_files(io_files[0], io_files[1], io_files[2], io_files[3], 'ani_salt_dummpy.pst')

# print(os.chdir(".."))
io_files

C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_conc_ca_009_day.txt.ins, trying inschek: list index out of range
C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_conc_ca_012_day.txt.ins, trying inschek: list index out of range


error using inschek for instruction file salt_conc_ca_009_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_ca_012_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.


C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_conc_ca_075_day.txt.ins, trying inschek: list index out of range
C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_conc_cl_012_day.txt.ins, trying inschek: list index out of range


error using inschek for instruction file salt_conc_ca_075_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_cl_012_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.


C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_conc_cl_075_day.txt.ins, trying inschek: list index out of range
C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_conc_co3_075_day.txt.ins, trying inschek: list index out of range


...

error using inschek for instruction file salt_load_so4_012_mon.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_load_so4_075_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.


C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_load_so4_075_day.txt.ins, trying inschek: list index out of range
C:\Users\seonggyu.park\AppData\Roaming\Python\Python37\site-packages\pyemu\pst\pst_utils.py:910: UserWarning: error processing instruction file salt_load_so4_075_mon.txt.ins, trying inschek: list index out of range


error using inschek for instruction file salt_load_so4_075_mon.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_ca_009_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_ca_012_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_ca_075_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_cl_012_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_conc_cl_075_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instructi

error using inschek for instruction file salt_load_na_012_mon.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_load_na_075_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_load_na_075_mon.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_load_so4_009_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_load_so4_009_mon.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruction file salt_load_so4_012_day.txt.ins:run() returned non-zero: 50
observations in this instruction file will havegeneric values.
error using inschek for instruc

(['EM_Animas.salt.tpl', 'rt3d_initc.par.tpl', 'salt_input.tpl'],
 ['EM_Animas.salt', 'rt3d_initc.par', 'salt_input'],
 ['salt_conc_ca_009_day.txt.ins',
  'salt_conc_ca_012_day.txt.ins',
  'salt_conc_ca_075_day.txt.ins',
  'salt_conc_cl_012_day.txt.ins',
  'salt_conc_cl_075_day.txt.ins',
  'salt_conc_co3_075_day.txt.ins',
  'salt_conc_hco3_012_day.txt.ins',
  'salt_conc_hco3_075_day.txt.ins',
  'salt_conc_k_009_day.txt.ins',
  'salt_conc_k_012_day.txt.ins',
  'salt_conc_k_075_day.txt.ins',
  'salt_conc_mg_009_day.txt.ins',
  'salt_conc_mg_012_day.txt.ins',
  'salt_conc_mg_075_day.txt.ins',
  'salt_conc_na_009_day.txt.ins',
  'salt_conc_na_012_day.txt.ins',
  'salt_conc_na_075_day.txt.ins',
  'salt_conc_so4_009_day.txt.ins',
  'salt_conc_so4_012_day.txt.ins',
  'salt_conc_so4_075_day.txt.ins',
  'salt_load_ca_009_day.txt.ins',
  'salt_load_ca_009_mon.txt.ins',
  'salt_load_ca_012_day.txt.ins',
  'salt_load_ca_012_mon.txt.ins',
  'salt_load_ca_075_day.txt.ins',
  'salt_load_ca_075_mon.txt

In [7]:
# load the pre-constructed pst
# pst = pyemu.Pst(os.path.join(wd, 'ani_salt_dummpy.pst'))

In [8]:
par = pst.parameter_data
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,extra
parnme,,,,,,,,,,,
aquf_caco3,aquf_caco3,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
aquf_caso4,aquf_caso4,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
aquf_mgso4,aquf_mgso4,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
init_ca,init_ca,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
init_cl,init_cl,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
init_cno3,init_cno3,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
init_co3,init_co3,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
init_hco3,init_hco3,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN
init_k,init_k,log,factor,1.0,1.100000e-10,1.100000e+10,pargp,1.0,0.0,1,NaN


In [9]:
for i in range(len(par)):
    if (par.iloc[i, 0][:2]) == 'aq':
        par.iloc[i, 6] = 'aquf'
    elif par.iloc[i, 0][:2] == 'in':
        par.iloc[i, 6] = 'initc'
    elif par.iloc[i, 0][:2] == 'so':
        par.iloc[i, 6] = 'soil'


In [12]:
par.index.rename('parnme1', inplace=True)

In [13]:
par = par.sort_values(by=['pargp', 'parnme'])
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,extra
parnme1,,,,,,,,,,,
aquf_caco3,aquf_caco3,log,factor,1.0,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1,NaN
aquf_caso4,aquf_caso4,log,factor,1.0,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1,NaN
aquf_mgso4,aquf_mgso4,log,factor,1.0,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1,NaN
init_ca,init_ca,log,factor,1.0,1.100000e-10,1.100000e+10,initc,1.0,0.0,1,NaN
init_cl,init_cl,log,factor,1.0,1.100000e-10,1.100000e+10,initc,1.0,0.0,1,NaN
init_cno3,init_cno3,log,factor,1.0,1.100000e-10,1.100000e+10,initc,1.0,0.0,1,NaN
init_co3,init_co3,log,factor,1.0,1.100000e-10,1.100000e+10,initc,1.0,0.0,1,NaN
init_hco3,init_hco3,log,factor,1.0,1.100000e-10,1.100000e+10,initc,1.0,0.0,1,NaN
init_k,init_k,log,factor,1.0,1.100000e-10,1.100000e+10,initc,1.0,0.0,1,NaN


In [14]:
for i in range(len(par)):
    if par.iloc[i, 6] == 'initc':
        par.iloc[i, 3] = 50.0001   # initial    
        par.iloc[i, 4] = 0.0001   # lower
        par.iloc[i, 5] = 100   # upper
        par.iloc[i, 8] = -50   # offset
    elif par.iloc[i, 6] == 'sy':
        par.iloc[i, 3] = 1.000000e-01 
        par.iloc[i, 4] = 1.000000e-03
        par.iloc[i, 5] = 0.800000e+00  
    elif par.iloc[i, 6] == 'hk':
        par.iloc[i, 3] = 1.000000e-01 
        par.iloc[i, 4] = 1.000000e-02
        par.iloc[i, 5] = 5.000000e+02
    elif par.iloc[i, 6] == 'rivcd':
        par.iloc[i, 3] = 50.01   # initial    
        par.iloc[i, 4] = 0.1   # lower
        par.iloc[i, 5] = 100   # upper
        par.iloc[i, 8] = -50   # offset

In [15]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,extra
parnme1,,,,,,,,,,,
aquf_caco3,aquf_caco3,log,factor,1.0000,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1,NaN
aquf_caso4,aquf_caso4,log,factor,1.0000,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1,NaN
aquf_mgso4,aquf_mgso4,log,factor,1.0000,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1,NaN
init_ca,init_ca,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1,NaN
init_cl,init_cl,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1,NaN
init_cno3,init_cno3,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1,NaN
init_co3,init_co3,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1,NaN
init_hco3,init_hco3,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1,NaN
init_k,init_k,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1,NaN


In [ ]:
# k01 = [87, 88, 92, 93, 94, 95]
# k01 = ['hk{:04d}'.format(i) for i in k01]
# for i in par.parnme:
#     for j in k01:
#         if i == j:
#             par.loc[i, 'parval1'] = 0.1
#             par.loc[i, 'parlbnd'] = 0.1* 0.01
#             par.loc[i, 'parubnd'] = 0.1* 100
            

In [16]:
obd = pst.observation_data
obd

,obsnme,obsval,weight,obgnme,extra
obsnme,,,,,
d009_cca_19961119,d009_cca_19961119,1.000000e+10,1.0,obgnme,NaN
d009_cca_19970108,d009_cca_19970108,1.000000e+10,1.0,obgnme,NaN
d009_cca_19970130,d009_cca_19970130,1.000000e+10,1.0,obgnme,NaN
d009_cca_19970225,d009_cca_19970225,1.000000e+10,1.0,obgnme,NaN
d009_cca_19970325,d009_cca_19970325,1.000000e+10,1.0,obgnme,NaN
...,...,...,...,...,...
m075_lso4_200906,m075_lso4_200906,1.000000e+10,1.0,obgnme,NaN
m075_lso4_200912,m075_lso4_200912,1.000000e+10,1.0,obgnme,NaN
m075_lso4_201005,m075_lso4_201005,1.000000e+10,1.0,obgnme,NaN


In [17]:
for i in range(len(obd)):
    if (obd.iloc[i, 0][5:9]) == 'lso4' or (obd.iloc[i, 0][5:9]) == 'cso4':
        obd.iloc[i, 3] = obd.iloc[i, 0][:9]
    elif (obd.iloc[i, 0][5:10]) == 'lhco3' or (obd.iloc[i, 0][5:10]) == 'chco3':
        obd.iloc[i, 3] = obd.iloc[i, 0][:10]
    elif (obd.iloc[i, 0][5:7]) == 'lk' or (obd.iloc[i, 0][5:7]) == 'ck':
        obd.iloc[i, 3] = obd.iloc[i, 0][:7]
    else:
        obd.iloc[i, 3] = obd.iloc[i, 0][:8]


In [32]:
print(obd)

                              obsnme        obsval  weight     obgnme  extra
obsnme                                                                      
d009_cca_19961119  d009_cca_19961119  1.000000e+10     1.0   d009_cca    NaN
d009_cca_19970108  d009_cca_19970108  1.000000e+10     1.0   d009_cca    NaN
d009_cca_19970130  d009_cca_19970130  1.000000e+10     1.0   d009_cca    NaN
d009_cca_19970225  d009_cca_19970225  1.000000e+10     1.0   d009_cca    NaN
d009_cca_19970325  d009_cca_19970325  1.000000e+10     1.0   d009_cca    NaN
...                              ...           ...     ...        ...    ...
m075_lso4_200906    m075_lso4_200906  1.000000e+10     1.0  m075_lso4    NaN
m075_lso4_200912    m075_lso4_200912  1.000000e+10     1.0  m075_lso4    NaN
m075_lso4_201005    m075_lso4_201005  1.000000e+10     1.0  m075_lso4    NaN
m075_lso4_201007    m075_lso4_201007  1.000000e+10     1.0  m075_lso4    NaN
m075_lso4_201111    m075_lso4_201111  1.000000e+10     1.0  m075_lso4    NaN

## 2.3. Import measured data

In [30]:
obd_files

['salt_009_day.obd',
 'salt_009_mon.obd',
 'salt_012_day.obd',
 'salt_012_mon.obd',
 'salt_075_day.obd',
 'salt_075_mon.obd']

In [39]:
ts = ['day', 'mon']
subids = [9, 12, 75]
obd_files_sort = []
for t in ts:
    for s in subids:
        obd_files_sort.append("salt_{:03d}_{}.obd".format(s, t))

In [44]:
obd_files_sort

['salt_009_day.obd',
 'salt_012_day.obd',
 'salt_075_day.obd',
 'salt_009_mon.obd',
 'salt_012_mon.obd',
 'salt_075_mon.obd']

In [31]:
# column names
colnams

['load_so4',
 'load_ca',
 'load_mg',
 'load_na',
 'load_k',
 'load_cl',
 'load_co3',
 'load_hco3',
 'conc_so4',
 'conc_ca',
 'conc_mg',
 'conc_na',
 'conc_k',
 'conc_cl',
 'conc_co3',
 'conc_hco3']

In [41]:
colnam_sort = colnams

In [42]:
colnam_sort.sort()

In [43]:
colnam_sort

['conc_ca',
 'conc_cl',
 'conc_co3',
 'conc_hco3',
 'conc_k',
 'conc_mg',
 'conc_na',
 'conc_so4',
 'load_ca',
 'load_cl',
 'load_co3',
 'load_hco3',
 'load_k',
 'load_mg',
 'load_na',
 'load_so4']

In [ ]:
# get total list from each sub obd, delete na vals
tot_obd = []
tot_obd += tot_fdc_obd
for j in lai_obd.columns:
    tot_obd += lai_obd.loc[:, j].dropna().tolist()    
len(tot_obd)

In [52]:
tot_obd = []
for obdf in obd_files_sort:
    salt_obd = pd.read_csv(
                        obdf,
                        sep='\t',
#                         usecols=['date', col_name],
                        index_col=0,
                        parse_dates=True,
                        na_values=[-999, '']
                        )
    salt_obd = salt_obd[cali_start_day:cali_end_day]
    salt_obd = salt_obd[~salt_obd.index.duplicated(keep='first')]
    for caln in colnam_sort:
        if caln in salt_obd.columns:
            tot_obd += salt_obd.loc[:, caln].dropna().tolist()  
            
            

In [53]:
len(tot_obd)

4486

In [54]:
obd.loc[:, 'obsval'] = tot_obd
obd

,obsnme,obsval,weight,obgnme,extra
obsnme,,,,,
d009_cca_19961119,d009_cca_19961119,52.4,1.0,d009_cca,NaN
d009_cca_19970108,d009_cca_19970108,56.1,1.0,d009_cca,NaN
d009_cca_19970130,d009_cca_19970130,69.2,1.0,d009_cca,NaN
d009_cca_19970225,d009_cca_19970225,60.1,1.0,d009_cca,NaN
d009_cca_19970325,d009_cca_19970325,44.8,1.0,d009_cca,NaN
...,...,...,...,...,...
m075_lso4_200906,m075_lso4_200906,220707.0,1.0,m075_lso4,NaN
m075_lso4_200912,m075_lso4_200912,130089.0,1.0,m075_lso4,NaN
m075_lso4_201005,m075_lso4_201005,248012.0,1.0,m075_lso4,NaN


# 4. Export control file

In [56]:
pst.parameter_data = par
pst.control_data.noptmax=0
pst.model_command = 'python salt_forward_run.py'
pst.write('ani_salt_pest2.pst')

noptmax:0, npar_adj:23, nnz_obs:4486


In [26]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom
aquf_caco3,aquf_caco3,log,factor,1.0000,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1
aquf_caso4,aquf_caso4,log,factor,1.0000,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1
aquf_mgso4,aquf_mgso4,log,factor,1.0000,1.100000e-10,1.100000e+10,aquf,1.0,0.0,1
init_ca,init_ca,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1
init_cl,init_cl,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1
init_cno3,init_cno3,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1
init_co3,init_co3,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1
init_hco3,init_hco3,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1
init_k,init_k,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1
init_mg,init_mg,log,factor,50.0001,1.000000e-04,1.000000e+02,initc,1.0,-50.0,1


In [57]:
os.getcwd()

'D:\\Projects\\Watersheds\\Elizabeth\\RBF_PS0G_cal_220614'